In [ ]:
from dotenv import load_dotenv
import os
import sys
import random

import numpy as np
import torch


current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)

sys.path.append(parent_dir)

load_dotenv()

SEED = 1337

In [ ]:
torch.use_deterministic_algorithms(True)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('mps') if torch.backends.mps.is_available() else torch.device('cpu')
print('Device:', device)

In [ ]:
def set_seed():
    random.seed(SEED)
    np.random.seed(SEED)
    
    torch.manual_seed(SEED)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(SEED)
        torch.cuda.manual_seed_all(SEED)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed()

In [ ]:
from src.data_loader import load_datasets

labelled_dev, unlabelled_dev, val_set = load_datasets("../data/")
train_df = labelled_dev
y_train = train_df['label']

test_df = val_set
y_test = test_df['label']

# Embedding

## TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

def fit_tf_idf_vectorizer(train_df, test_df):
    vectorizer = TfidfVectorizer(stop_words='english', max_features=10000)
    X_train_tfidf = vectorizer.fit_transform(train_df['content'])
    X_test_tfidf = vectorizer.transform(test_df['content'])
    return X_train_tfidf, X_test_tfidf

X_train_tfidf, X_test_tfidf = fit_tf_idf_vectorizer(train_df, test_df)

## BERT with Average Pooling

In [ ]:
from transformers import AutoTokenizer, BertModel
import torch

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased", force_download=True)

model = BertModel.from_pretrained("google-bert/bert-base-uncased", force_download=True)
model.to(device)


def get_bert_embedding(text):
    inputs = tokenizer(text, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        last_hidden_states = outputs.last_hidden_state
        return last_hidden_states

def get_sentence_embedding(text):
    with torch.no_grad():
        last_hidden_states = get_bert_embedding(text)
        return torch.mean(last_hidden_states, dim=1).squeeze()
    
def fit_bert_vectorizer(train_df, test_df):
    X_train_bert = torch.stack([get_sentence_embedding(text) for text in train_df['content']])
    X_test_bert = torch.stack([get_sentence_embedding(text) for text in test_df['content']])
    return X_train_bert, X_test_bert

#X_train_bert, X_test_bert = fit_bert_vectorizer(train_df, test_df)

## Sentence Transformers


In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')
model.to(device)
def generate_embeddings(texts):
    embeddings = model.encode(texts, show_progress_bar=True)
    return embeddings

# Generate embeddings for a small subset due to computational limits
X_train_sentence_transformer = generate_embeddings(train_df['content'].values)
X_test_sentence_transformer = generate_embeddings(test_df['content'].values)


## KNeighbors Weak Labeling

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=40)
knn.fit(X_train_sentence_transformer, y_train)

In [ ]:
y_pred = knn.predict(X_test_sentence_transformer)

In [ ]:
from sklearn.metrics import classification_report, accuracy_score

print(classification_report(test_df['label'], y_pred))
print(f"Accuracy: {accuracy_score(test_df['label'], y_pred)}")

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_neighbors': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 200],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski']
}

grid_search = GridSearchCV(KNeighborsClassifier(), 
                           param_grid, 
                           n_jobs=-1, 
                           verbose=3)

grid_search.fit(X_train_sentence_transformer, y_train)

In [ ]:
import os
import pickle

MODELS_FOLDER = os.getenv('MODELS_DIR', 'models')
os.makedirs(MODELS_FOLDER, exist_ok=True)

with open(f'{MODELS_FOLDER}/wl_knn.pkl', 'wb') as f:
    pickle.dump(grid_search, f)

In [ ]:
with open(f'{MODELS_FOLDER}/wl_knn.pkl', 'rb') as f:
    knn = pickle.load(f)

In [ ]:
print('--- Best KNN Parameters ---')
print(f'Parameters: {grid_search.best_params_}')
print(f'Score: {grid_search.best_score_:.2f}')

knn_best = knn.best_estimator_
knn_best.fit(X_train_sentence_transformer, y_train)
y_pred = knn_best.predict(X_test_sentence_transformer)

In [ ]:
from src.px_utils import create_dataset, launch_px

test_ds = create_dataset("Test Dataset", test_df, X_test_sentence_transformer.tolist(), y_pred)
train_ds = create_dataset("Train Dataset", train_df, X_train_sentence_transformer.tolist(), y_train)

launch_px(test_ds, train_ds)